# Backup your data

This script will backup your poweropti data by iteratively downloading all the data from the API.

Note that only historical data (energy consumption in kWh) can be backed up effectively with a time resolution of 15-min aggregated values.

In [ ]:
from powerfox_api import PowerfoxAPI, JSON
from pathlib import Path
from datetime import date, timedelta, datetime

# the ones you used in your app …
USERNAME = "" # <- your username here
PASSWORD = "" # <- your password here

assert USERNAME and PASSWORD, "Please provide your Powerfox credentials"

timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
FILENAME = Path(f"./poweropti_backup_{timestamp}.jsonl") # <- uses JSON Lines 

api = PowerfoxAPI(
    username=USERNAME,
    password=PASSWORD,
)

Crucial settings for the backup. Make sure to adjust `device` if you have multiple devices.

In [ ]:
devices = api.get_devices()
device = devices[0]
start = device.account_associated_since.date()
end = date.today()
# we want to scrape each data
# this allows us to get the maximum time resolution
# (i.e., if we would scrape each month instead, data would be more aggresively aggregated)
dates = [start + timedelta(days=day) for day in range((end - start).days)]


BATCH_APPEND = 15  # append data in batches of 15 days
RETRIES = 10
BACKOFF = 2  # exponential backoff in seconds

The following backups all your historical energy consumption day by day with 15-min aggregation resolution. When tested, it took about 2 minutes per 365 days.

In [ ]:
from time import sleep
from json import dumps

def backup_date(day: date):
    retries = 0
    while retries < RETRIES:
        try:
            hisotrical_data = api.get_historical_data_raw(device_id=device.id, day=day)
            return hisotrical_data
        except Exception as e:
            print(f"Error while fetching data for {day}: {e}")
            retries += 1
            sleep(BACKOFF * retries)

    print(f"Failed to fetch data for {day} after {RETRIES} retries")

def append_to_json_file(batch: dict[date, JSON], *, file: Path):
    with file.open("a") as f:
        for day, data in batch.items():
            f.write(
                dumps({
                    "day": day.isoformat(),
                    "data": data
                })
                + "\n"
            )

def backup(date_range: list[date] = dates, *, file: Path = FILENAME):
    batch: dict[date, JSON] = {}

    for day in date_range:
        data = backup_date(day)

        if data is None:
            continue

        batch[day] = data

        if len(batch) >= BATCH_APPEND:
            append_to_json_file(batch, file=file)
            batch = {}

    # dump the remaining
    if len(batch) > 0:
        append_to_json_file(batch, file=file)


In [ ]:
# start the backup
backup()